In [1]:
%load_ext autoreload
%autoreload 2
!pip install caafe

In [2]:
from caafe import CAAFEClassifier # Automated Feature Engineering for tabular datasets
from tabpfn import TabPFNClassifier # Fast Automated Machine Learning method for small tabular datasets
from sklearn.ensemble import RandomForestClassifier

import os
import openai
import torch
from caafe import data
from sklearn.metrics import accuracy_score
from tabpfn.scripts import tabular_metrics
from functools import partial
from caafe.preprocessing import make_datasets_numeric

# CatDB packages
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss

/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# ```python
# Load the training and test datasets
train_data = pd.read_csv('../data/balance-scale/balance-scale_train.csv')
test_data = pd.read_csv('../data/balance-scale/balance-scale_test.csv')
# ```end
target = 'class'


# ```python
# Perform data cleaning and preprocessing
# As per the given schema, there are no missing values and all the columns are of integer type. So, no data cleaning is required.
# ```end

# ```python
# Perform feature processing
# Scale the numerical columns
scaler = MinMaxScaler()
train_data[['right-distance', 'left-distance', 'right-weight', 'left-weight']] = scaler.fit_transform(train_data[['right-distance', 'left-distance', 'right-weight', 'left-weight']])
test_data[['right-distance', 'left-distance', 'right-weight', 'left-weight']] = scaler.transform(test_data[['right-distance', 'left-distance', 'right-weight', 'left-weight']])
# ```end

# ```python
# Select the appropriate features and target variables
features = ['right-distance', 'left-distance', 'right-weight', 'left-weight']

train_data, test_data = make_datasets_numeric(train_data, test_data, target)

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


In [11]:
### Setup Base Classifier

clf = TabPFNClassifier(device=('cuda' if torch.cuda.is_available() else 'cpu'), N_ensemble_configurations=4)
clf.fit = partial(clf.fit, overwrite_warning=True)

clf.fit(X_train, y_train)
# pred = clf.predict(X_test)
# acc = accuracy_score(pred, y_test)
# print(f'Accuracy before CAAFE {acc}')
#TabPFN-balance-scale

/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


TabPFNClassifier(N_ensemble_configurations=4)

In [12]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

Train_Accuracy = accuracy_score(y_train, y_train_pred)
Test_Accuracy = accuracy_score(y_test, y_test_pred)

Train_Log_loss = log_loss(y_train, clf.predict_proba(X_train))
Test_Log_loss = log_loss(y_test, clf.predict_proba(X_test))

print(f"Train_Accuracy:{Train_Accuracy}")   
print(f"Train_Log_loss:{Train_Log_loss}") 
print(f"Test_Accuracy:{Test_Accuracy}")   
print(f"Test_Log_loss:{Test_Log_loss}")

/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/saeed/Documents/Github/CatDB/envCatDB/lib/python3.10/site-packages/pandas/core/algorithms.py:1814: Depr

Train_Accuracy:1.0
Train_Log_loss:0.0610072340978834
Test_Accuracy:0.9375
Test_Log_loss:0.138097871969876
